In [1]:
import os
import PyPDF2
import markdown2
import json
import pickle
import pandas as pd

from docx import Document as DocxDocument
from pptx import Presentation

from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings.base import OpenAIEmbeddings

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Rinats well documented sample project
# https://github.com/trustbit/private-poc-fmw-content-generator/tree/main/backend-and-kb/src/fmw

from dotenv import load_dotenv

load_dotenv()

True

## Get Companies

In [4]:
SAMPLES_PATH = r"C:\Users\felix.krause\code\trustbit\enterprise-rag-challenge\samples"

In [5]:
df = pd.read_csv("data/dataset.csv") 
df

,sha1,date,name,size
0,ce9e5024041b2ece2bafa2a9d9516bb174ee8949,2022-10-31,"Anixa Biosciences, Inc.",3996701
1,f71415f9ca0cff70e5fa193616b6197f361130ed,2023-02-21,"Maravai LifeSciences Holdings, Inc.",4033642
2,4a9d2b853e05970776121a810460f0962a18c5a1,2022-XX,KLA Corporation,1181894
3,f973dd219c534accb0d4e72d8e12f51284d48d10,2023-01-01,"Ameresco, Inc.",10648267
4,4e27f4c3402c657d548760cb3a164b036cefaabb,2022-12-31,Battery Minerals Limited,3650701
...,...,...,...,...
7490,a2afcd8165a6dbd0058682680b65d3638a5800eb,2023-02-02,"Arrow Electronics, Inc",922750
7491,215df84494756bd4feebc973657835ef7f14ee16,2022-12-31,Synertec,10474497
7492,20fb970d8705289e835b408c575351295ac16f5f,2022-09-30,TE Connectivity,5753097
7493,c7f3a8c0a38c756438950ce3085076adb4241a32,2022-01-01,OTC_ADDDF,12603054


In [6]:
# load all file names in the folder
sha1_dict_full = df.set_index("sha1").to_dict()["name"]

included_companies = []

for file in os.listdir(SAMPLES_PATH):
    file = file[:-4]
    try:
        included_companies.append(sha1_dict_full[file])
    except KeyError:
        print(file)

questions.


In [7]:
included_companies

['Global Medical REIT Inc.',
 'Zegona Communications plc',
 'Calyxt, Inc.',
 'TD SYNNEX',
 'Eurocell PLC',
 'Sandwell Aquatics Centre',
 'Mercurity Fintech Holding Inc.',
 'Motus GI Holdings, Inc.',
 'Limbach Holdings, Inc.',
 'Strike Energy Limited',
 'Platform Technology',
 'VENUS METALS CORPORATION LIMITED',
 'NICE',
 'Nykredit',
 'Air Products',
 'Accuray Incorporated',
 'Kooth Plc',
 'Ethernity Networks Ltd',
 'Canadian Banc Corp.',
 'EMT']

In [8]:
# Create sample for development
# COMPANIES = ["Ethernity Networks Ltd", "Limbach Holdings, Inc.", "Accuray Incorporated"]
COMPANIES = included_companies

if COMPANIES:
    df_sample = df[df.name.isin(COMPANIES)]
else:
    df_sample = df.copy()
    
sha1_dict = df_sample.set_index("name").to_dict()["sha1"]

In [9]:
df_sample

,sha1,date,name,size
267,d81bbc64a4160b9946fea7a895f80e6201f52f27,2022-11-23,Air Products,4889969
981,608c5097dfc6e83505fd2259ad862dcec11a3f96,2022-12-31,Sandwell Aquatics Centre,2661906
1075,3696c1b29566acc1eafc704ee5737fb3ae6f3d1d,2023-01-01,Zegona Communications plc,1481844
1183,99be213e4e689294ebae809bfa6a1b5024076286,2022-01-01,"Limbach Holdings, Inc.",2023552
1324,71b04e0248ecf758990a0ab77bd69344be63bcf4,2023-03-27,"Motus GI Holdings, Inc.",7725729
1500,6b79f1c1de9d0e39a4576dcd4585849b9465b402,2023-04-05,Mercurity Fintech Holding Inc.,1767320
2633,40b5cfe0d7bbf59e186492bfbe1b5002d44af332,2023-03-01,"Calyxt, Inc.",694201
2838,faf8d7d79152d61279eda1cfb58b8236ce2f82fa,2022-06-30,EMT,1360293
3270,4b525836a5d7cb75489f6d93a3b1cf2b8f039bf2,2022-10-01,TD SYNNEX,1902884
4263,ea0757d27fa67cd347d9f046b939a911f5c9a08d,2022-01-01,Canadian Banc Corp.,498408


## Get company name from user query

TODO could be more sophisticated

In [10]:
import re
from fuzzywuzzy import fuzz

def normalize_name(name):
    # Normalize by removing extra spaces and converting to lowercase
    return re.sub(r'\s+', ' ', name.strip().lower())

def fuzzy_match(companies, input_string, threshold=85):
    matched_companies = []
    norm_input = normalize_name(input_string)
    
    for company in companies:
        norm_company = normalize_name(company)
        prev_token = ""
        
        # Fuzzy match each token
        for curr_token in norm_input.split():
            token = prev_token + " " + curr_token
            
            match_score = fuzz.ratio(norm_company, token)
            prev_token = curr_token
            
            if match_score >= threshold:
                matched_companies.append(company)
                break  # Stop once a match is found for this company
            
    return matched_companies


def find_exact_matches(companies, input_string):
    # normalized_input = normalize_name(input_string)
    
    matched_companies = []
    
    for company in companies:
        if company in input_string:
            matched_companies.append(company)
    
    return matched_companies


def find_companies(input_string):
    # Wrapper to use later and allow easy function change
    return find_exact_matches(df.name, input_string)


input_string = ("limbach holding inc and ethernity networks buys the accuracy incorporated, ESEA, Five9, Inc.")

matches = find_exact_matches(df.name, input_string)
print(matches)

['EA', 'ESEA', 'Five9, Inc.']


C:\Users\felix.krause\code\trustbit\llm_experiments\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Load LLM data

In [11]:
from llama_parse import LlamaParse

parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True
)

# Function to load text from different file types
def load_text_from_file(file_path):
    _, file_extension = os.path.splitext(file_path)
    text = ""
    
    if file_extension == ".pdf":
        # TODO https://www.reddit.com/r/LangChain/comments/18yxacm/extracting_data_from_pdf_containing_complex_tables/
        with open(file_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f, strict=False)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    else:
        pass
    
    return text


# Function to load documents from a folder
def load_documents_from_folder(folder_path, companies=[]):
    documents = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file[:-4] in companies: #or len(companies) == 0:
                print("Loading", file)
                file_path = os.path.join(root, file)

                text = load_text_from_file(file_path)
                if text:
                    documents.append(Document(page_content=text, metadata={"source": file_path}))
    return documents

In [12]:
# Could try LLama approach
# TODO https://medium.com/the-ai-forum/rag-on-complex-pdf-using-llamaparse-langchain-and-groq-5b132bd1f9f3
# doc = parser.load_data(os.path.join(FOLDER_PATH, "e51b7204b91cbe7709bd3218e7d2d0c2b8dbb438.pdf"))
# index = VectorStoreIndex.from_documents(doc)

## Try to detect tables from pdf

In [8]:
"""
import camelot

# Extract tables from a PDF file
tables = camelot.read_pdf(os.path.join(FOLDER_PATH, "e51b7204b91cbe7709bd3218e7d2d0c2b8dbb438.pdf"), pages="all")

# Convert the first table to a DataFrame
df = tables[0].df

# Save the DataFrame as a CSV file
tables[0]
"""

C:\Users\felix.krause\code\trustbit\llm_experiments\venv\lib\site-packages\camelot\parsers\lattice.py:397: UserWarning: No tables found on page-2
  warnings.warn(
C:\Users\felix.krause\code\trustbit\llm_experiments\venv\lib\site-packages\camelot\parsers\lattice.py:397: UserWarning: No tables found on page-79
  warnings.warn(


<Table shape=(1, 1)>

In [47]:
# tables[40].df

,0,1,2,3
0,STRATEGIC REPORT\nCORPORATE GOVERNANCE,,FINANCIAL STATEMENTS\nFINANCIAL STATEMENTS,73\n73


## Build retriever per company

In [13]:
def get_store_name(string):
    string = string.lower()
    return string.replace(" ", "_").replace(",", "").replace(".", "").replace("'", "")

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200) # TODO more sophisticated

for company in df_sample.name.to_list():
    # check if file already exists
    if os.path.exists(f"dbs/{get_store_name(company)}.db"):
        print("Skipping", company)
        continue
    
    try:
        company_docs = load_documents_from_folder(SAMPLES_PATH, [sha1_dict[company]])
        
        if not company_docs:
            raise ValueError(f"No documents found for company: {company}")
        
        texts = text_splitter.split_documents(company_docs)
        embeddings = OpenAIEmbeddings()
        db = FAISS.from_documents(texts, embeddings)
        
        with open(f"dbs/{get_store_name(company)}.db", "wb") as f:
            pickle.dump(db.serialize_to_bytes(), f)
    except Exception as e:
        raise e
        print(f"Error for company {company}: {e}")

Skipping Air Products
Skipping Sandwell Aquatics Centre
Skipping Zegona Communications plc
Skipping Limbach Holdings, Inc.
Skipping Motus GI Holdings, Inc.
Skipping Mercurity Fintech Holding Inc.
Skipping Calyxt, Inc.
Skipping EMT
Skipping TD SYNNEX
Skipping Canadian Banc Corp.
Loading 9e703e719d94af786af5511c823ff86e9f04c070.pdf
Skipping Kooth Plc
Skipping Ethernity Networks Ltd
Skipping NICE
Skipping Eurocell PLC
Skipping VENUS METALS CORPORATION LIMITED
Skipping Accuray Incorporated
Skipping Nykredit
Loading 9ae3bb21564a5098c4b4d6450655c22eff85deae.pdf
Skipping Global Medical REIT Inc.


In [15]:
def get_retriever_dict(companies):
    retriever_dict = {}
    
    for company in companies:
        with open(f"dbs/{get_store_name(company)}.db", "rb") as f:
            db_bytes = pickle.load(f)
        
        db_temp = FAISS.deserialize_from_bytes(db_bytes, OpenAIEmbeddings(),
                                      allow_dangerous_deserialization=True)
        
        retriever_dict[company] = db_temp.as_retriever()
    
    return retriever_dict

In [16]:
retriever_dict = get_retriever_dict(df_sample.name.to_list())

In [25]:
def get_context(query, company):
    # TODO add sophisticated retriever for tables?
    return retriever_dict[company].get_relevant_documents(query)

## Structured inference

In [18]:
MODEL = "gpt-4o-mini-2024-07-18"

# load system prompt from .md
with open("instructions/answer_guidelines.md") as f:
    ANSWER_GUIDELINES = f.read()
    
with open("instructions/fin_info.md") as f:
    FIN_INFO = f.read()



system_prompt = (
    f"You are an intelligent assistant tasked with answering questions based on specific company data provided in the context. "
    
    f"Please provide a chain of thought on how you arrived at your final answer in 'chain_of_thought', providing intermediary results. \n"
    f"Then give a final answer in 'answer' where you strictly adhere to the following guidelines: \n {ANSWER_GUIDELINES} \n "
    f"Ensure that your answers are strictly compliant with these guidelines. Accuracy and adherence to the format are crucial. \n"
    
    # f"Your primary task is to accurately identify and extract specific financial metrics, ratios, and counts, even when synonymous "
    # f"or contextually similar terms are used. Here is some useful information about these: \n {FIN_INFO}"
)

In [33]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class AnswerEvent(BaseModel):
    chain_of_thought: str
    answer: int | str


def ask_company_expert(query, company):
    print("# ASKING COMPANY EXPERT:", company)
    context = get_context(query, company)
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt + f"Context: \n {context}"},
            {"role": "user", "content": query},
        ],
        response_format=AnswerEvent,
    )
    
    return completion.choices[0].message.parsed

In [43]:
ask_company_expert("How many assets does 'Accuray Incorporated' have in 2022 in dollars?", "Accuray Incorporated")

['Accuray Incorporated']


AnswerEvent(chain_of_thought="The total assets for 'Accuray Incorporated' in 2022 are stated in the consolidated balance sheets provided. The relevant value for total assets in 2022 is given as $472,849 (in thousands). To convert this into dollars, we multiply this figure by 1,000. Thus, the total assets for 'Accuray Incorporated' in 2022 amount to 472849000 dollars.", answer=472849000)

In [96]:
ask_company_expert("How many assets does 'Ethernity Networks Ltd' have in 2022 in dollars?", "Ethernity Networks Ltd")

['Ethernity Networks Ltd']


AnswerEvent(chain_of_thought='The provided context does not contain specific information regarding the total assets of Ethernity Networks Ltd for the year 2022. Therefore, I cannot provide an answer as the necessary data is absent.', answer='n/a')

In [69]:
ask_company_expert("What are the total R&D expenses of 'Ethernity Networks Ltd' in 2021?", "Ethernity Networks Ltd")

['Ethernity Networks Ltd']


AnswerEvent(chain_of_thought='In the provided context, it is mentioned that the total R&D expenses for Ethernity Networks Ltd in 2021 were 5,550,912.', answer=5550912)

## Multiple agents

In [36]:
# DELEGATION MANAGER

INVOLVED_COMPANIES = df_sample.name.to_list()

system_prompt_delegation = (
    f"You will receive questions about one or more companies. "
    f"You have access to specialised agents in providing data for each company. "
    f"To access them, you need to provide the exact company name in a list of 'companies' and the respective query in a list of 'queries'. \n"
    f"For example, if the question is about if company X has more assets than company Y in 2022, you would provide the following: \n"
    f"companies: ['X', 'Y'], queries: ['How many assets does 'X' have in 2022 in dollars?', 'How many assets does 'Y' have in 2022 in dollars?'] \n"
    f"The company name in the list has to be exactly one of these: {INVOLVED_COMPANIES}. \n"
    f"Also provide a chain of thought on how you arrived at your final answer in 'chain_of_thought'. \n"
    f"If the company asked is not in the list, write in companies and queries: 'SKIP' and make a note in the 'chain_of_thought'. \n"
    f"Consider however, that queries can contain incomplete company names, for example skipping 'Inc.' or 'Ltd.' \n"
)


class DelegationRequest(BaseModel):
    chain_of_thought: str
    companies: list[str]
    queries: list[str]


def ask_delegation_manager(query):    
    print("# ASKING DELEGATION MANAGER: ")
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_delegation},
            {"role": "user", "content": query},
        ],
        response_format=DelegationRequest,
    )
    
    return completion.choices[0].message.parsed

In [46]:
ask_delegation_manager("Has 'Ethernity Networks Ltd' more liabilities than 'Accuray Incorporated' in 2022?")

ManagerRequest(chain_of_thought="To determine if 'Ethernity Networks Ltd' has more assets than 'Accuray Incorporated' in 2022, I need to compare their total assets for that year. Therefore, I will first query the total assets of 'Ethernity Networks Ltd' for 2022 and then do the same for 'Accuray Incorporated'. This will allow me to compare the two values directly.", companies=['Ethernity Networks Ltd', 'Accuray Incorporated'], queries=["How many assets does 'Ethernity Networks Ltd' have in 2022 in dollars?", "How many assets does 'Accuray Incorporated' have in 2022 in dollars?"])

In [45]:
ask_delegation_manager("How many assets does 'Ethernity Networks Ltd' have in 2022 in dollars?")

ManagerRequest(chain_of_thought="To answer the question regarding the assets of 'Ethernity Networks Ltd' in 2022, I need to access the data provider concerning this specific company and ask for the relevant financial information. Herein, the query will be about the total assets it holds in 2022 in dollar value, which is a direct question about the company's financial stature in that year.", companies=['Ethernity Networks Ltd'], queries=["How many assets does 'Ethernity Networks Ltd' have in 2022 in dollars?"])

In [37]:
# EXECUTION MANAGER

system_prompt_execution = (
    f"You are an intelligent assistant tasked with answering questions based on specific company data provided in the context. "
    "Also the chain of thought of the expert providing the data is given. "
    "Your task is to provide a final answer to the USER_QUERY based on the data provided (CONTEXT). \n"
    
    f"Please provide a chain of thought on how you arrived at your final answer in 'chain_of_thought', providing intermediary results. \n"
    f"Then give a final answer in 'answer' where you strictly adhere to the following guidelines: \n {ANSWER_GUIDELINES} \n "
    f"Ensure that your answers are strictly compliant with these guidelines. Accuracy and adherence to the format are crucial. \n"
    
    "USER_QUERY: {QUERY} \n"
    
    "CONTEXT: {CONTEXT} \n"
)

class ExecutionRequest(BaseModel):
    chain_of_thought: str
    answer: int | str

def ask_execution_manager(query, context):
    system_prompt_execution_full = system_prompt_execution.replace("{QUERY}", query).replace("{CONTEXT}", context)
    
    print("# ASKING EXECUTION MANAGER: ")
    print("Query:", query)
    print("Context:", context)
    
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_execution_full},
            {"role": "user", "content": query},
        ],
        response_format=ExecutionRequest,
    )
    
    return completion.choices[0].message.parsed

In [38]:
# FINAL PIPELINE

def ask_question(query):
    delegations = ask_delegation_manager(query)
    print("Delegations: \n", delegations)
    expert_context = {}
    
    for company, company_query in zip(delegations.companies, delegations.queries):
        if company.lower() == "skip" or company_query.lower() == "skip":
            expert_context[company] = "No data available"
        else:
            expert_context[company] = ask_company_expert(company_query, company)
    
    final_answer = ask_execution_manager(query, str(expert_context))
    
    return final_answer

In [27]:
ask_question("Who is the CEO of Tesla Inc.?") # n/a

# ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="Tesla Inc. is not on the provided company list, so I cannot create a query for it. Instead, I will note this as 'SKIP'." companies=['SKIP'] queries=['SKIP']
# ASKING EXECUTION MANAGER: 
Query: Who is the CEO of Tesla Inc.?
Context: {'SKIP': 'No data available'}


ExecutionRequest(chain_of_thought="The context provided does not contain any information regarding the CEO of Tesla Inc. Since there are no available details, I cannot provide an answer to this query. Hence, I conclude that the answer is insufficient and will respond with 'n/a'.", answer='n/a')

In [39]:
ask_question("Who is the CEO of Ethernity Networks?") # David Levi

# ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought='Ethernity Networks is included in the list of companies provided, and the query about its CEO is straightforward. Therefore, I can construct the request using Ethernity Networks as the company and asking specifically for the CEO information.' companies=['Ethernity Networks Ltd'] queries=["Who is the CEO of 'Ethernity Networks'?"]
# ASKING COMPANY EXPERT
# ASKING EXECUTION MANAGER: 
Query: Who is the CEO of Ethernity Networks?
Context: {'Ethernity Networks Ltd': AnswerEvent(chain_of_thought='The document clearly states that David Levi is the Chief Executive Officer of Ethernity Networks. This role is directly mentioned under the section that lists company directors and their positions.', answer='David Levi')}


ExecutionRequest(chain_of_thought='The document clearly states that David Levi is the Chief Executive Officer of Ethernity Networks. This role is directly mentioned under the section that lists company directors and their positions.', answer='David Levi')

In [40]:
ask_question("Has 'Ethernity Networks Ltd' more liabilities than 'Accuray Incorporated' in 2022?") # Accuray: 419,660,000$, Ethernity: 12,257,291$ -> no is correct answer

# ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="I need to compare the liabilities of two companies: 'Ethernity Networks Ltd' and 'Accuray Incorporated'. Both company names are in the provided list, which allows me to create the necessary queries for each company. I'll ask for their liabilities in 2022, which will allow me to compare them directly." companies=['Ethernity Networks Ltd', 'Accuray Incorporated'] queries=["How many liabilities does 'Ethernity Networks Ltd' have in 2022 in dollars?", "How many liabilities does 'Accuray Incorporated' have in 2022 in dollars?"]
# ASKING COMPANY EXPERT
# ASKING COMPANY EXPERT
# ASKING EXECUTION MANAGER: 
Query: Has 'Ethernity Networks Ltd' more liabilities than 'Accuray Incorporated' in 2022?
Context: {'Ethernity Networks Ltd': AnswerEvent(chain_of_thought="To find the total liabilities for Ethernity Networks Ltd in 2022, we examine the provided financial statements data. We specifically look for any mention of 'total liabilities

ExecutionRequest(chain_of_thought='From the context provided, Ethernity Networks Ltd has total liabilities of 1121909 dollars in 2022. On the other hand, Accuray Incorporated has total liabilities of 419660 dollars for the same year. To determine if Ethernity Networks Ltd has more liabilities, we compare the two figures: 1121909 is greater than 419660. Therefore, the conclusion is that Ethernity Networks Ltd has more liabilities than Accuray Incorporated in 2022.', answer='yes')

## Obtain final results 

In [41]:
results = []

with open('data/questions.json', 'r') as json_file:
    items = json.load(json_file)
    for item in items:
        question = item["question"]
        print(question + "\n")
        response = ask_question(question) # TODO could introduce schema as Daniel with arg=item["schema"] and classes/pydantic
        print(response)
        results.append({
            "question": question,
            "schema": item["schema"],
            "answer": response.answer
        })
        

with open('results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

How much did "Accuray Incorporated" spend on risk management in Q2 2022?

# ASKING DELEGATION MANAGER: 
Delegations: 
 chain_of_thought="The query specifically asks about 'Accuray Incorporated' and is within the permissible company names. I can directly form the queries to request the relevant financial information about risk management spending for that specific quarter." companies=['Accuray Incorporated'] queries=["How much did 'Accuray Incorporated' spend on risk management in Q2 2022 in dollars?"]
# ASKING COMPANY EXPERT
# ASKING EXECUTION MANAGER: 
Query: How much did "Accuray Incorporated" spend on risk management in Q2 2022?
Context: {'Accuray Incorporated': AnswerEvent(chain_of_thought="The provided context does not specify expenditures by 'Accuray Incorporated' related to risk management or specifically mention financial amounts allocated to risk management activities for Q2 2022. Hence, as I cannot find any relevant data pertaining to that spending within the context given, I

In [42]:
results

[{'question': 'How much did "Accuray Incorporated" spend on risk management in Q2 2022?',
  'schema': 'number',
  'answer': 'n/a'},
 {'question': 'Who is the CEO in the company "Zegona Communications plc"?',
  'schema': 'name',
  'answer': 'Eamonn O’Hare'},
 {'question': 'Did "Global Medical REIT Inc." have a greater Debt-to-Equity ratio than "Zegona Communications plc" in Q2 2021?',
  'schema': 'boolean',
  'answer': 'n/a'},
 {'question': 'How many stores did "Accuray Incorporated" have in the end of fiscal year 2021?',
  'schema': 'number',
  'answer': 'n/a'},
 {'question': 'How much did "Sandwell Aquatics Centre" spend on R&D in Q2 2023?',
  'schema': 'number',
  'answer': 'n/a'},
 {'question': 'Who is the CFO in the company "EMT"?',
  'schema': 'name',
  'answer': 'n/a'},
 {'question': 'Did "Calyxt, Inc." have a greater Return on Assets (ROA) than "Global Medical REIT Inc." in Q2 2023?',
  'schema': 'boolean',
  'answer': 'n/a'},
 {'question': 'How many stores did "OTC_UNCIF" have 

## OLD Inference with chain OLD

In [7]:
# Create vector store
# https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/
# Load documents from the specified folder
documents = load_documents_from_folder(SAMPLES_PATH, df_sample.sha1.to_list())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) # TODO more sophisticated
texts = text_splitter.split_documents(documents)

if len(texts) == 0:
    raise ValueError("No text found in the specified folder")

In [8]:
embeddings = OpenAIEmbeddings() # deprecated?
db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever()

In [54]:
# Store vector database
# with open("data/first_sample_db_3", "wb") as f:
#     pickle.dump(db.serialize_to_bytes(), f)

In [8]:
# Load vector database
# with open("data/vector_db_uni", "rb") as f:
#     db_bytes = pickle.load(f)
#     db = FAISS.deserialize_from_bytes(db_bytes, OpenAIEmbeddings(),
#                                       allow_dangerous_deserialization=True)
#     retriever = db.as_retriever()

In [31]:
llm = ChatOpenAI(temperature=0)
# llm = llm.bind(logprobs=True, top_logprobs=3)
# could try local models as well

system_prompt_2 = (
    f"You are an intelligent assistant tasked with answering questions based on specific company data provided in the context. "
    f"Please provide a chain of thought on how you arrived at your final answer in 'chain_of_thought'. \n"
    f"Then give a final answer in 'answer' where you strictly adhere to the following guidelines: \n {ANSWER_GUIDELINES} \n "
    f"Ensure that your answers are strictly compliant with these guidelines. Accuracy and adherence to the format are crucial. \n"
    
    # f"Your primary task is to accurately identify and extract specific financial metrics, ratios, and counts, even when synonymous "
    # f"or contextually similar terms are used. Here is some useful information about these: \n {fin_info}"
    
    "Context: \n {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt_2),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)


# Function to ask a question
def ask_question(query):
    # retriever = get_retriever(query)
    
    chain = create_retrieval_chain(retriever, question_answer_chain)
    
    response = chain.invoke({"input": query})
    answer = response["answer"]
    print(f"Question: {query}\nAnswer: {answer}")

```python
    ('What was the {fin_metric} of "{company}" in {time_frame}?', "number"),
    ('How much did "{company}" spend on {focus_area} in {time_frame}?', "number"),
    ('What was the {ratio_or_metric} of "{company}" in {time_frame}?', "number"),
    ('How many {count_metric} did "{company}" have in {time_frame}?', "number"),
    ('Which company had a higher {fin_metric}: "{company1}", "{company2}" or "{company3}", in {time_frame}?', "name"),
    ('Did "{company1}" have a greater {ratio_or_metric} than "{company2}" in {time_frame}?', "boolean"),
    ('How much more did "{company1}" spend on {focus_area} compared to "{company2}" in {time_frame}?', "number"),
    ('Who is the {role} in the company "{company}"?', "name"),
```

In [ ]:
# multiple agents
# retriever per company
# -> first check if we need several processes to answer the questions (e.g. check revenues of 3 companies, then use these values with a final agent)
# chain of though: output json with chain of thought and final answer

In [28]:
# Ask a question
ask_question("How many assets does 'Accuray Incorporated' have in 2022 in dollars?") # 472.849 

Question: How many assets does 'Accuray Incorporated' have in 2022 in dollars?
Answer: chain_of_thought: 
1. Look for the Consolidated Balance Sheets section to find the total assets for Accuray Incorporated in 2022.
2. Identify the value listed under "Total current assets" and any other asset categories to calculate the total assets for the company in 2022.

answer: 352890


In [29]:
ask_question("How many assets does 'Accuray Incorporated' have in 2021 in dollars?") # 480.098

Question: How many assets does 'Accuray Incorporated' have in 2021 in dollars?
Answer: Chain of Thought:
1. Look for the Consolidated Balance Sheets section.
2. Find the total assets value for 'Accuray Incorporated' for June 30, 2021.

Answer:
352773


In [32]:
ask_question("How many liabilities does 'Accuray Incorporated' have in 2021 in dollars?") # 411.258

Question: How many liabilities does 'Accuray Incorporated' have in 2021 in dollars?
Answer: Chain of Thought:
1. Liabilities can be calculated by subtracting total assets from total equity, as liabilities = assets - equity.
2. The total assets for 2021 are not provided directly but can be calculated by summing up the current assets and other assets like property, equipment, etc.
3. The total equity for 2021 is not directly given but can be calculated by adding common stock, additional paid-in capital, accumulated other comprehensive income (loss), and accumulated deficit from the stockholders' equity statement.
4. Once total assets and total equity are calculated, liabilities can be determined.

Answer:
$378,000

Chain of Thought:
- Total assets for 2021 are calculated as $350,890 (current assets) + $12,685 (property and equipment) + $13,879 (investment in joint venture) + $16,798 (operating lease right-of-use assets) + $57,840 (goodwill) + $250 (intangible assets) = $452,342.
- Total 

In [78]:
ask_question("How many stores did 'Strike Energy Limited' have in the end of fiscal year 2021?") # n/a

Question: How many stores did 'Strike Energy Limited' have in the end of fiscal year 2021?
Answer: n/a


In [83]:
# Total R&D Expenses of Ethernity Networks
ask_question("What are the total R&D expenses of 'Ethernity Networks' in 2021?") # 5 550 912

ask_question("What are the total R&D expenses of 'Ethernity Networks' in 2022?") # 6 618 795

Question: What are the total R&D expenses of 'Ethernity Networks' in 2021?
Answer: 5550912
Question: What are the total R&D expenses of 'Ethernity Networks' in 2022?
Answer: 6618795
